In [20]:
import pyrealsense2 as rs

In [21]:
deviceDetect = False
ctx = rs.context()
devices = ctx.query_devices()

for dev in devices:
    productName = str(dev.get_info(rs.camera_info.product_id))
    print(productName)
    
    # DS 435 config
    if productName in "0B07":
        EnableColorStream = True
        EnableInfraredStream = True
        EnableDepthStream = True
        deviceDetect = True
        break
    
    # DS 415 config
    elif productName in "0AD3":
        EnableColorStream = True
        EnableInfraredStream = True
        EnableDepthStream = True
        deviceDetect = True
        break
        
    # DS 410 config
    elif productName in "0AD2":
        EnableColorStream = False
        EnableInfraredStream = True
        EnableDepthStream = True
        deviceDetect = True
        break

if deviceDetect is not True:
    raise Exception("No supported device was found")

0B07


In [22]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

if EnableColorStream is True:
    config.enable_stream(rs.stream.color)

if EnableInfraredStream is True:
    config.enable_stream(rs.stream.infrared, 1)
    config.enable_stream(rs.stream.infrared, 2)
    
if EnableDepthStream is True:
    config.enable_stream(rs.stream.depth)
    
# Start streaming
cfg = pipeline.start(config)

# Get Video Stream Intrinsics
print("=======================================================")
print("Get Video Stream Intrinsics")
print("=======================================================")
colorStream = cfg.get_stream(rs.stream.color)
colorProfile = colorStream.as_video_stream_profile()
colorIntrin = colorProfile.get_intrinsics()
print("Color Intrinsics:\n  width:" , colorIntrin.width, "\n  height:" , colorIntrin.height, "\n  ppx:" , colorIntrin.ppx, "\n  ppy:" , colorIntrin.ppy, "\n  fx:" , colorIntrin.fx, "\n  fy:" , colorIntrin.fy, "\n  model:" , colorIntrin.model)
print("")

infraredStream = cfg.get_stream(rs.stream.infrared, 1)
infraredProfile = infraredStream.as_video_stream_profile()
infraredIntrin = infraredProfile.get_intrinsics()
print("Infrared Intrinsics:\n  width:" , infraredIntrin.width, "\n  height:" , infraredIntrin.height, "\n  ppx:" , infraredIntrin.ppx, "\n  ppy:" , infraredIntrin.ppy, "\n  fx:" , infraredIntrin.fx, "\n  fy:" , infraredIntrin.fy, "\n  model:" , infraredIntrin.model)

# Get and Apply Depth-to-Stream Extrinsics
print("\n=======================================================")
print("Get and Apply Depth-to-Stream Extrinsics")
print("=======================================================")
print("Color Extrinsics:")
depthStream = cfg.get_stream(rs.stream.depth)
colorExtrin = depthStream.get_extrinsics_to(colorStream)
print(colorExtrin)
print("")
print("Infrared Extrinsics:")
infraredExtrin = depthStream.get_extrinsics_to(infraredStream)
print(infraredExtrin)

# Get Disparity Baseline
print("\n=======================================================")
print("Get Disparity Baseline")
print("=======================================================")
infraredStream2 = cfg.get_stream(rs.stream.infrared, 2)
baseline = infraredStream.get_extrinsics_to(infraredStream2)
print(baseline)

Get Video Stream Intrinsics
Color Intrinsics:
  width: 640 
  height: 480 
  ppx: 310.5052490234375 
  ppy: 245.05064392089844 
  fx: 618.7859497070312 
  fy: 617.9658203125 
  model: distortion.brown_conrady

Infrared Intrinsics:
  width: 1280 
  height: 720 
  ppx: 639.13232421875 
  ppy: 362.58013916015625 
  fx: 637.5574340820312 
  fy: 637.5574340820312 
  model: distortion.brown_conrady

Get and Apply Depth-to-Stream Extrinsics
Color Extrinsics:
rotation: [0.999963, 0.0072977, -0.00455206, -0.00728499, 0.99997, 0.00280114, 0.00457237, -0.00276788, 0.999986]
translation: [0.0145757, 0.00041163, 0.000190269]

Infrared Extrinsics:
rotation: [1, 0, 0, 0, 1, 0, 0, 0, 1]
translation: [0, 0, 0]

Get Disparity Baseline
rotation: [1, 0, 0, 0, 1, 0, 0, 0, 1]
translation: [-0.0499424, 0, 0]


In [23]:
pipeline.stop()